# Compare Flight Positions

Compare positions between a "reference" trajectory and another trajectory.

In [ ]:
import numpy as np
import pandas as pd
import folium
import matplotlib.pyplot as plt
%matplotlib inline

from pru.EcefPoint import rad2nm
from pru.EcefPath import EcefPath
from pru.ecef_functions import calculate_EcefPoints
from pru.trajectory_functions import calculate_elapsed_times, \
    find_duplicate_values
from pru.horizontal_path_functions import derive_horizontal_path
from pru.trajectory_analysis import DEFAULT_ACROSS_TRACK_TOLERANCE, \
    calculate_ground_speeds, smooth_times

## Get reference positions file
Examples:  
- RYR234R_positions_2017-09-27.csv  
- RYR5KK_positions_2017-09-28.csv  
- RYR9TJ_positions_2017-09-28.csv  
- RYR7UR_positions_2017-09-29.csv

In [ ]:
ref_filename = input('Ref filename: ')
ref_points_df = pd.read_csv(ref_filename, parse_dates=['TIME'])
len(ref_points_df)

## Calculate the Horizontal Path

In [ ]:
ecef_points = calculate_EcefPoints(ref_points_df['LAT'].values,
                                   ref_points_df['LON'].values)

In [ ]:
across_track_tolerance = DEFAULT_ACROSS_TRACK_TOLERANCE
threshold=np.deg2rad(DEFAULT_ACROSS_TRACK_TOLERANCE / 60.0)
ecef_path = derive_horizontal_path(ecef_points,
                                   threshold=threshold)
len(ecef_path)

In [ ]:
ecef_path_distances = rad2nm(ecef_path.path_distances())
ecef_path_length = ecef_path_distances[-1]
ecef_path_length

## Plot Horizontal Path

The reference points are in Black, the horizontal path is Blue.

In [ ]:
point_locations = ref_points_df[['LAT','LON']].values.tolist()

waypoint_locations = []
for point in ecef_path.points:
    waypoint_locations.append(point.to_lat_long())
#waypoint_locations

In [ ]:
# Create the map
map_mb = folium.Map(location=[50.0, 10.0], zoom_start=5, tiles='Cartodb Positron')

BLACK = '#000000'
BLUE = '#0000FF'
RED = '#FF0000'

# Add a PolyLine to show point_locations
folium.PolyLine(
    locations=point_locations,
    color=BLACK,
    weight=2,
    tooltip='raw points',
).add_to(map_mb)

# Add a PolyLine to show waypoint_locations
folium.PolyLine(
    locations=waypoint_locations,
    color=BLUE,
    weight=2,
    tooltip='extreme points',
).add_to(map_mb)

# Display the map
map_mb

## Calculate reference distances

Calculate reference point distances along the horizontal path. 

In [ ]:
ref_path_distances = rad2nm(ecef_path.calculate_path_distances(ecef_points, threshold))
len(ref_path_distances)

## Get other positions file
Examples:  
- RYR234R_cpr_positions_2017-09-27.csv
- RYR234R_fr24_positions_2017-09-27.csv
- RYR5KK_cpr_fr24_positions_2017-09-28.csv
- RYR9TJ_cpr_positions_2017-09-28.csv
- RYR9TJ_fr24_positions_2017-09-28.csv
- RYR7UR_cpr_fr24_positions_2017-09-29.csv

In [ ]:
filename = input('Other filename: ')
points_df = pd.read_csv(filename, parse_dates=['TIME'])
len(points_df)

## Plot Points relative to Path

In [ ]:
point_lat_lons = points_df[['LAT','LON']].values.tolist()

In [ ]:
# Add a PolyLine to show point_locations
folium.PolyLine(
    locations=point_lat_lons,
    color=RED,
    weight=2,
    tooltip='raw points',
).add_to(map_mb)

# Add a PolyLine to show waypoint_locations
#folium.PolyLine(
#    locations=waypoint_locations,
#    color=BLUE,
#    weight=2,
#    tooltip='extreme points',
#).add_to(map_mb)

# Display the map
map_mb

## Calculate distances relative to reference path

In [ ]:
ecef_points = calculate_EcefPoints(points_df['LAT'].values,
                                   points_df['LON'].values)

In [ ]:
path_distances = rad2nm(ecef_path.calculate_path_distances(ecef_points, threshold))
len(path_distances)

In [ ]:
# Sort positions by path distance then time
sorted_df = pd.DataFrame({'distance': path_distances,
                          'time': points_df['TIME'].values,
                          'altitude': points_df['ALT'].values,
                          'points': ecef_points})
sorted_df.sort_values(by=['distance', 'time'], inplace=True)

sorted_path_distances = sorted_df['distance'].values

## Display Vertical Profiles

Plot the vertical profiles as altiude vs path distance.

In [ ]:
ref_alts = ref_points_df['ALT'].values
len(ref_alts)

In [ ]:
alts = sorted_df['altitude'].values
len(alts)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))
ax.set(title='Vertical Profile',
       xlabel='Path Distance (NM)',
       ylabel='Altitude (feet)')
plt.plot(ref_path_distances, ref_alts, 'o-')
plt.plot(sorted_path_distances, alts, 'o-')
#for i in range(len(ref_path_distances)):
#    ax.annotate(str(i), (ref_path_distances[i], alts[i] +500))
plt.show

## Calculate Ground Speeds

Calculate ground speeds between positions from the 
path distances and elapsed times from the first position.

In [ ]:
ref_speeds = ref_points_df['SPEED_GND'].values

In [ ]:
duplicate_positions = find_duplicate_values(sorted_path_distances,
                                            across_track_tolerance)
valid_distances = sorted_path_distances[~duplicate_positions]

In [ ]:
times = sorted_df['time'].values
elapsed_times = calculate_elapsed_times(times[~duplicate_positions], times[0])
len(elapsed_times)

In [ ]:
speeds = calculate_ground_speeds(valid_distances, elapsed_times)  
len (speeds)

## Plot Ground Speeds

Plot ground speeds vs path distance.

Note: the first speed is always zero.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))
ax.set(title='Speed Profile',
       xlabel='Path Distance (NM)',
       ylabel='Ground Speed (Knots)')
plt.plot(ref_path_distances, ref_speeds, 'o-')
# plt.plot(valid_distances, speeds, 'o-')
#for i in range(len(sorted_path_distances)):
#    ax.annotate(str(i), (sorted_path_distances[i], speeds[i] +10))
plt.show

## Smooth Times and Ground Speeds

Use the smooth_times to smooth the elapsed_times.

In [ ]:
smoothed_times = smooth_times(valid_distances, elapsed_times)
len(smoothed_times)

In [ ]:
# Calculate differences from elapsed_times
delta_times = smoothed_times - elapsed_times
# delta_times

### Calculate the mean time difference

In [ ]:
mean_delta = np.sum(delta_times) / len(delta_times)
mean_delta

### Calculate the time Standard Deviation

In [ ]:
# Offset the smoothed_times by the mean time difference
smoothed_times = smoothed_times - mean_delta

sq_delta_times = (smoothed_times - elapsed_times) ** 2
time_sd = np.sqrt(np.sum(sq_delta_times) / (len(sq_delta_times) - 1))
time_sd

In [ ]:
smoothed_speeds = calculate_ground_speeds(valid_distances, smoothed_times)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))
ax.set(title='Speed Profile',
       xlabel='Path Distance (NM)',
       ylabel='Ground Speed (Knots)')
plt.plot(ref_path_distances, ref_speeds, 'o-')
plt.plot(valid_distances, smoothed_speeds, 'o-')
#for i in range(len(sorted_path_distances)):
#    ax.annotate(str(i), (sorted_path_distances[i], smoothed_speeds[i] +10))
plt.show